In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('Report_JR14_Vehicle History Report - data.xlsx',skiprows=10)

In [5]:
'''
    Create Trips from the GPS Data
'''

df.columns

Index(['GPS', 'Device', 'Server', 'Reason', 'Driver', 'Desc.', 'Rego',
       'Latitude', 'Longitude', 'Direction', 'Unnamed: 10', 'Speed', 'RawGPS',
       'Status', 'Flags', 'User Defined', 'Input', 'Output', 'Light', 'Button',
       'Max', 'Average', 'Spd Acc', 'Samples', 'Map Ref', 'Suburb', 'Distance',
       'Position', 'Eng Cool Temp', 'Eng Oil Temp', 'OilPressure',
       'ECM Economy', 'Trip', 'Total Used', 'RPM', 'Tot Hour', 'Brake Hits',
       'Odometer', 'MaxF', 'MaxB', 'MaxLR', 'MaxZ', 'Position.1', '  1  ',
       '  2', '  3  ', '  4  ', '  5  ', '  6  ', '  7  ', '  8  ', '  9  ',
       '  10  ', 'Volts', 'Seconds', 'Speed.1', 'Zone 1', 'Zone 2', 'Zone 3',
       'Zone 4', 'Zone 1.1', 'Zone 2.1', 'Zone 3.1', 'Zone 4.1', '  1  .1',
       '  2  ', '  3  .1', '  4  .1', '  5  .1', 'Vehicle', 'Auxiliary',
       '  1  .2', '  2  .1', '  3  .2', '  4  .2', '  5  .2', 'Mass', 'Limit',
       'HDOP', 'Satellites', 'Temperature', 'From Flash', 'Offset Time',
       'Offset'

In [ ]:
'''
    Initial Data Sanitation on Odometer Reading.

    Filter the data from the date where Odometer Reading > 500.
'''
df['Date'] = pd.to_datetime(df['GPS'],format="%Y-%m-%d").dt.date
reasons = df.groupby(by='Date')['Reason'].unique()

In [ ]:
'''
    Read and process a list of dates for calendar file.
 
'''

def create_calendar_file(df):
    dates = df['GPS'].dt.date.unique()

    calendardates = [
        {'date': item, 'day_of_week': item.strftime('%A')} for item in dates
    ]
    calendar_cols = ['service_id','start_date','end_date','monday','tuesday','wednesday','thursday','friday','saturday','sunday']
    calendar = pd.DataFrame(columns=calendar_cols)

    for date in calendardates:
        day = date['day_of_week']
        idate = date['date'].strftime('%Y%m%d')
        if  day == 'Monday':
            data = pd.DataFrame([[idate,idate,idate,'1','0','0','0','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data],ignore_index=True)
        elif day == 'Tuesday':
            data = pd.DataFrame([[idate,idate,idate,'0','1','0','0','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Wednesday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','1','0','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Thursday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','1','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Friday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','0','1','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Saturday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','0','0','1','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Sunday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','0','0','0','1']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)

    if len(calendar) != len(dates):
        print('Some dates have not been exported to Calendar')
    else:
        return calendar
    
print(create_calendar_file(df))


In [11]:
'''
    Data Processing:
    1. Filter required columns
    2. Concat Lat+Long into coords for each row
'''

def filter_by_columns(df,cols):
    return df[cols]

df = filter_by_columns(df,['GPS', 'Device', 'Server', 'Reason', 'Driver', 'Desc.','Speed', 'Rego','Latitude', 'Longitude', 'Direction','Odometer', 'MaxF', 'MaxB', 'MaxLR', 'MaxZ'])

df['coords'] = df['Latitude'].astype(str) + ','+ df['Longitude'].astype(str)

In [ ]:
'''
    Create Routes from Trips
'''
